<a href="https://colab.research.google.com/github/mitra369/210112-CN_CIFAR10/blob/main/210112_CN_CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
os.listdir("210112-CN_CIFAR10/dataset/phone image")


FileNotFoundError: [Errno 2] No such file or directory: '210112-CN_CIFAR10/dataset/phone image'

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


In [ ]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),
                         (0.5, 0.5, 0.5))
])


In [ ]:
trainset = datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform)

testset = datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform)


In [ ]:
classes = ['bird', 'car', 'cat', 'dog']
class_map = [2, 1, 3, 5]  # CIFAR10 indexes


In [ ]:
def filter_dataset(dataset):
    idx = [i for i, (_, label) in enumerate(dataset) if label in class_map]
    dataset.data = dataset.data[idx]
    dataset.targets = [class_map.index(dataset.targets[i]) for i in idx]
    return dataset


In [ ]:
trainset = filter_dataset(trainset)
testset = filter_dataset(testset)


In [ ]:
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
testloader = DataLoader(testset, batch_size=64, shuffle=False)


In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 4)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = torch.relu(self.fc1(x))
        return self.fc2(x)


In [ ]:
model = CNN().to(device)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
train_loss, train_acc = [], []

for epoch in range(10):
    correct, total, running_loss = 0, 0, 0
    model.train()

    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss.append(running_loss / len(trainloader))
    train_acc.append(correct / total)

    print(f"Epoch {epoch+1} | Loss {train_loss[-1]:.4f} | Acc {train_acc[-1]*100:.2f}%")


In [ ]:
torch.save(model.state_dict(), "model/210112.pth")
